In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
import tensorflow as tf
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import hinge_loss, mean_absolute_error,mean_squared_log_error,median_absolute_error,mean_absolute_percentage_error,mean_poisson_deviance

In [3]:
file_path = '../dataset.json'

# Open the file and load its contents
with open(file_path, 'r') as json_file:
    # Read the file content
    file_content = json_file.read()

    # Split the content by newline to get individual JSON objects
    json_objects = file_content.split('\n')

data = []
# Process each JSON object
for json_object in json_objects:
    if json_object.strip() == '':
        continue  # Skip empty lines
    
    # Parse the JSON object
    data.append(json.loads(json_object))

print(len(data))

X = []
Y = []
Y_flat = []

for i in data:
    X.append(np.array(i['vector']))    
    Y.append(np.array(i['matrix']))
    
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.4, random_state=44)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
Y_train_flat =[i.flatten() for i in Y_train]
Y_test_flat =[i.flatten() for i in Y_test]

1411


In [4]:

def get_metrics(y, p_y):
    metrics = {}
    try:
        metrics['mse'] = mse(y,p_y)
    except:
        pass
    try:
        metrics['hinge_loss'] = hinge_losss(y,p_y)
    except:
        pass
    try:
        metrics['mean_absolute_error'] = mean_absolute_error(y,p_y)
    except:
        pass
    try:
        metrics['mean_squared_log_error'] = mean_squared_log_error(y,p_y)
    except:
        pass
    try:
        metrics['median_absolute_error'] = median_absolute_error(y,p_y)
    except:
        pass
    try:
        metrics['mean_absolute_percentage_error'] = mean_absolute_percentage_error(y,p_y)
    except:
        pass
    try:
        metrics['mean_poisson_deviance'] = mean_poisson_deviance(y,p_y)
    except:
        pass

    return metrics
    

# Handmade NN

In [5]:
# Define the neural network architecture
def build_crazy_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(2048, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(output_dim)
    ])
    return model
def build_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(196)  # Output layer with shape (196,)
    ])
    return model

def build_model_recursive(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Reshape((1, 256)),  # Reshape input to match LSTM requirements
        # tf.keras.layers.LSTM(512, return_sequences=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='relu')),
        tf.keras.layers.LSTM(512),
        tf.keras.layers.Dense(output_dim)
    ])
    return model

# def build_model_convolutional(input_dim):
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
#         tf.keras.layers.Dense(7*7*128, activation='relu'),
#         tf.keras.layers.Reshape((7, 7, 128)),
#         tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2DTranspose(2, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2D(1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Reshape((14, 14, 1))  # Update the output shape to (14, 14, 1)
#     ])
#     return model

In [6]:
# Define the input and output dimensions
input_dim = 128  # Dimensionality of the input vector
output_dim = 196  # Dimensionality of the output vector

In [7]:
# Build the model
model = build_model_recursive(input_dim, output_dim)
model_ann = build_model(input_dim, output_dim)
# model_conv = build_model_convolutional(input_dim)

# Compile the model
model.compile(optimizer='adam', loss='mse')
model_ann.compile(optimizer='adam', loss='mse')
# model_conv.compile(optimizer='adam', loss='mse')

In [8]:
model_ann.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=30)

Epoch 1/30
27/27 [==============================] - 2s 27ms/step - loss: 1789813.0000
Epoch 2/30
27/27 [==============================] - 1s 27ms/step - loss: 1535.7041
Epoch 3/30
27/27 [==============================] - 1s 27ms/step - loss: 1525.1367
Epoch 4/30
27/27 [==============================] - 1s 27ms/step - loss: 1757.0555
Epoch 5/30
27/27 [==============================] - 1s 27ms/step - loss: 1603.8358
Epoch 6/30
27/27 [==============================] - 1s 27ms/step - loss: 1520.1294
Epoch 7/30
27/27 [==============================] - 1s 28ms/step - loss: 1520.1285
Epoch 8/30
27/27 [==============================] - 1s 28ms/step - loss: 1520.1277
Epoch 9/30
27/27 [==============================] - 1s 27ms/step - loss: 1520.1268
Epoch 10/30
27/27 [==============================] - 1s 27ms/step - loss: 1520.1256
Epoch 11/30
27/27 [==============================] - 1s 27ms/step - loss: 1520.1251
Epoch 12/30
27/27 [==============================] - 1s 27ms/step - loss: 1520.123

In [9]:
model.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=30)

Epoch 1/30
27/27 [==============================] - 3s 14ms/step - loss: 1519.7838
Epoch 2/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.5587
Epoch 3/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.4749
Epoch 4/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.3931
Epoch 5/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.2053
Epoch 6/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.1143
Epoch 7/30
27/27 [==============================] - 0s 14ms/step - loss: 1518.9363
Epoch 8/30
27/27 [==============================] - 0s 14ms/step - loss: 1518.8491
Epoch 9/30
27/27 [==============================] - 0s 14ms/step - loss: 1518.7942
Epoch 10/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.1202
Epoch 11/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.0084
Epoch 12/30
27/27 [==============================] - 0s 14ms/step - loss: 1518.9216
E

In [10]:
Y_predicted = model.predict(np.array(X_test))
Y_ann_predicted = model_ann.predict(np.array(X_test))
# Y_conv_predicted = model_conv.predict(np.array(X_test))
print(get_metrics(Y_predicted, Y_test_flat))
print(get_metrics(Y_ann_predicted, Y_test_flat))

18/18 [==============================] - 0s 3ms/step
{'accuracy': 0.0, 'mse': 74.35198830079388, 'mean_absolute_error': 0.13910764401594042, 'median_absolute_error': 0.09361839123572552, 'mean_absolute_percentage_error': 2.546053918175268}
{'accuracy': 0.0, 'mse': 74.68787309301749, 'mean_absolute_error': 0.06742639544786548, 'median_absolute_error': 0.006336912573711696, 'mean_absolute_percentage_error': 15.515945562618311}


In [11]:
Y_predicted = [np.reshape(i, (14,14)) for i in Y_predicted]
# Y_predicted

In [12]:
Y_ann_predicted = [np.reshape(i, (14,14)) for i in Y_ann_predicted]
# Y_ann_predicted

# AutoKeras

In [13]:
import autokeras as ak
import numpy as np

In [14]:
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
train_set = tf.data.Dataset.from_tensor_slices((X_train, Y_train_flat))
test_set = tf.data.Dataset.from_tensor_slices((X_test, Y_test_flat))


In [15]:

input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=False)(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=100,metrics=['mse'],
)
reg.fit(train_set,epochs=15)

# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)|

Trial 53 Complete [00h 00m 04s]
val_loss: 217.4120635986328

Best val_loss So Far: 211.0436553955078
Total elapsed time: 00h 03m 42s
INFO:tensorflow:Oracle triggered exit
Epoch 1/15
27/27 [==============================] - 2s 3ms/step - loss: 1520.5531 - mse: 1520.5531
Epoch 2/15
27/27 [==============================] - 0s 3ms/step - loss: 1518.2599 - mse: 1518.2599
Epoch 3/15
27/27 [==============================] - 0s 3ms/step - loss: 1516.9558 - mse: 1516.9558
Epoch 4/15
27/27 [==============================] - 0s 3ms/step - loss: 1515.7854 - mse: 1515.7854
Epoch 5/15
27/27 [==============================] - 0s 3ms/step - loss: 1514.7783 - mse: 1514.7783
Epoch 6/15
27/27 [==============================] - 0s 3ms/step - loss: 1515.7646 - mse: 1515.7646
Epoch 7/15
27/27 [==============================] - 0s 3ms/step - loss: 1513.7859 - mse: 1513.7859
Epoch 8/15
27/27 [==============================] - 0s 3ms/step - loss: 1513.4513 - mse: 1513.4513
Epoch 9/15
27/27 [===================

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [16]:
predicted_y = reg.predict(test_set)
print(reg.evaluate(test_set))
print(get_metrics(Y_test_flat, predicted_y))
predicted_y

18/18 [==============================] - 0s 2ms/step - loss: 75.0349 - mse: 75.0349
[75.03491973876953, 75.03491973876953]
{'accuracy': 0.0, 'mse': 75.0349249143321, 'mean_absolute_error': 0.3475291074501781, 'median_absolute_error': 0.23809958837245734, 'mean_absolute_percentage_error': 98378864702.74512}


array([[-2.11254194e-01, -3.07110369e-01,  1.87819861e-02, ...,
        -1.57700226e-01,  1.01653171e+00,  5.53711548e+01],
       [-6.74944669e-02, -1.85701981e-01, -1.81589454e-01, ...,
        -3.26376669e-02, -4.98163439e-02,  1.04031515e+01],
       [-5.30624837e-02, -1.38605028e-01, -3.12669687e-02, ...,
        -1.66468009e-01,  4.20905769e-01,  1.15251598e+01],
       ...,
       [-1.15610018e-01, -2.00630933e-01, -7.42398500e-02, ...,
        -3.31348889e-02,  5.41591085e-02,  1.55125389e+01],
       [-1.79375961e-01, -3.74882281e-01,  6.52066171e-02, ...,
        -2.01920746e-04,  1.02803004e+00,  5.80619202e+01],
       [-1.86146632e-01, -2.59324849e-01, -3.22393812e-02, ...,
        -4.06718664e-02,  9.36430514e-01,  5.31868515e+01]], dtype=float32)

In [ ]:
regressor = ak.StructuredDataRegressor(max_trials=100, overwrite=True, metrics=['mse'],)
regressor.fit(train_set, epochs=11)
predicted_y = regressor.predict(test_set)
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
print(regressor.evaluate(test_set))
print(get_metrics(Y_test_flat, predicted_y))

Trial 82 Complete [00h 00m 09s]
val_loss: 213.0825958251953

Best val_loss So Far: 212.44825744628906
Total elapsed time: 00h 11m 37s

Search: Running Trial #83

Value             |Best Value So Far |Hyperparameter
True              |True              |structured_data_block_1/normalize
False             |False             |structured_data_block_1/dense_block_1/use_batchnorm
2                 |2                 |structured_data_block_1/dense_block_1/num_layers
32                |32                |structured_data_block_1/dense_block_1/units_0
0.5               |0                 |structured_data_block_1/dense_block_1/dropout
256               |256               |structured_data_block_1/dense_block_1/units_1
0                 |0                 |regression_head_1/dropout
adam              |adam              |optimizer
0.001             |0.001             |learning_rate
16                |16                |structured_data_block_1/dense_block_1/units_2



In [ ]:
model = reg.export_model()
model.summary()

In [ ]:
model_regressor = regressor.export_model()
model_regressor.summary()

In [ ]:
X_test = X_test.reshape(-1, 128)
Y_ak_predicted =reg.predict(X_test)

In [ ]:
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
train_set = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
test_set = tf.data.Dataset.from_tensor_slices((X_test, Y_test))


In [ ]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=100,metrics=['mse'],
)
reg.fit(train_set,epochs=15)

In [ ]:
predicted_y = reg.predict(test_set)
print(reg.evaluate(test_set))
print(get_metrics(Y_test_flat, predicted_y))
predicted_y